In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Embedding, LSTM, Bidirectional
from keras.optimizers import RMSprop, Adam, SGD
from keras import optimizers
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
from random import shuffle, sample
import datetime
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from random import shuffle, sample
import  matplotlib.pyplot as plt
from keras.utils import to_categorical
#create sample weights
from sklearn.utils import compute_sample_weight
from sklearn.utils import compute_class_weight
import os
os.chdir('/home/saul/bugpred')

In [23]:
maxlen = 5 #400 # number of words in a row. Input words.
batch_size = 200 #32 #32 #200 gave the best results
embedding_dims = 500 #300 #5 #300
epochs = 150 #best result is with 20
num_neurons = 251 #50
sample_size =  997 #10

In [24]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
import tensorflow as tf
import datetime

In [26]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [27]:
def getcbow(dataset):

    sentences = []
    vectorised_codes = []
    print("Cbow called")

    #bugs = pd.read_csv('bug-metrics.csv', sep= ',')
    #print(bugs.columns)

    ast = [row.split('::') for row in dataset['classname']]
    #print('ASTs ', ast[:2])
    #the imput to the cbow is list of list of each line
    #size of the word vector of a given token must be equal to embedding_dim of the LSTM model
    cbowmodel = Word2Vec(ast, min_count=1, size= embedding_dims, workers=3, window=3, sg=0)
    #print(ast[:2])
    print (' CBOW model ', cbowmodel)
    
    #Test cbow model
    print("Test CBOW on the data")
    print(cbowmodel['eclipse'])
    
    classes = dataset['classname']

    for codes in classes:

        linecode = []
        tokens = codes.split('::')
        #print(tokens)
        sentences.append(tokens)
        for token in tokens:
            try:
                #print("Token ", token)
                #linecode.append(token)
                #print("Word Vector ", len(cbowmodel[token]))
                linecode.append(cbowmodel[token])
            except KeyError:
                pass
        vectorised_codes. append(linecode)

    return vectorised_codes


In [28]:
def pad_trunc(data, maxlen):
    new_data = []
    zero_vector = []

    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(temp)
        else:
            temp = sample
        new_data.append(temp)

    return new_data

In [29]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen , embedding_dims))),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='sigmoid')  # two class
    
  ])

In [30]:
def lstmModel(vectorised_data, target):

    split_point =  int(len(vectorised_data) * .7)
    print('Split Point ', split_point)

    #split data into training and testing
    x_train = vectorised_data[:split_point]
    y_train = target[:split_point]

    x_test = vectorised_data[split_point:]
    y_test = target[split_point:]

    #make each point of data of uniform lenght
    x_train = pad_trunc(x_train, maxlen)
    x_test = pad_trunc(x_test, maxlen)

    #reshape data into a numpy structure
    
    print("X_TRAIN Reshape Started ")
    print(f' Training data Size: {len(x_train)}')
    print("Number of word tokens ", maxlen)
    print("Embedding Dims ", embedding_dims)


    x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
    print("X_TRAIN Reshape Completed ")
    #y_train = np.array(y_train)
    y_train = to_categorical(y_train, 2)

    x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
    #y_test = np.array(y_test)
    y_test = to_categorical(y_test, 2)
    #print(f'Y_TEST DATA: {y_test}')
    print((f'Y_TEST_DATA LENGHT{len(y_test)}'))
    print("Data Reshape Ended ")
    
    model = create_model()

    rmsprob = RMSprop(learning_rate=0.0001, rho=0.4) # use learning rate to improve the accuracy of the model
    adam = Adam(lr=0.001)
    #sgd = SGD(lr=0.1)
    #sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.2, nesterov=True)    

    model.compile(loss='binary_crossentropy', optimizer= rmsprob, metrics=['accuracy'])
    
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    fitmodel(model, x_train, y_train, x_test, y_test, batch_size, epochs, [tensorboard_callback])


In [31]:
def fitmodel(model, x_train, y_train, x_test, y_test, batch_size, epochs, tensorboard_callback):

    print('Y Test ', y_test.shape)

    cls_weight_dict = [{0: 1, 1: 1}, {0: 1, 1: 80}] #two class mapping of weights
    val_sample_weights = compute_sample_weight(cls_weight_dict, y_test)

    weights = compute_sample_weight(class_weight="balanced", y=y_train)

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              #sample_weight = weights,
              class_weight = {0 : 1. , 1: 80.},
              #class_weight={0: 1., 1: 100.},
              validation_data=(x_test, y_test, val_sample_weights),
              callbacks=[tensorboard_callback])


In [32]:
def getDataset():

    dataset = pd.read_csv('bug-metrics.csv', sep= ',')

    #keep = ['classname', 'bugs']
    dataset = dataset.sample(n= sample_size, replace= True, random_state=1)

    #shuffle(dataset)
    #dataset.to_csv('sampledataset.csv')
    return dataset


In [33]:
dataset = getDataset()
#getmaxlen(dataset)
vectorised_data = getcbow(dataset)

target = collect_expected(dataset) #Biased two classes {198, 2} lenght is 200
#print (target)
lstmModel(vectorised_data, target)

Cbow called
 CBOW model  Word2Vec(vocab=648, size=500, alpha=0.025)
Test CBOW on the data
[ 4.33608107e-02  9.74239632e-02  6.72205612e-02  1.75349936e-01
 -2.81045586e-02  1.40621200e-01 -1.42611712e-01  7.23332763e-02
  7.40287900e-02 -3.82003076e-02  3.80077660e-02 -7.58453384e-02
 -2.02103835e-02  9.86236185e-02  2.14544404e-02 -3.24227102e-02
 -2.98386123e-02  1.05249062e-01  5.18811122e-02 -1.39894694e-01
  9.17500816e-03 -1.25790432e-01  7.72704259e-02 -1.00175448e-01
  1.79270044e-01  1.06193079e-02 -8.35145358e-03  2.10743956e-02
 -1.18528508e-01 -4.27872390e-02  1.99522879e-02  1.60560876e-01
  1.14116684e-01  9.70501229e-02 -1.86513029e-02  8.15443322e-02
 -5.26925996e-02 -1.41300401e-02  1.35928512e-01  9.87130497e-03
  8.33240747e-02 -2.53285170e-02  5.10498695e-02 -7.52034932e-02
  1.84580521e-03 -1.02309927e-01  2.76369657e-02 -4.22766432e-02
  5.11343479e-02 -1.12836510e-02  3.58895510e-02  9.87377465e-02
  1.81819618e-01  1.27283931e-01 -1.26040980e-01 -4.48674113e-02


/home/saul/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/saul/.local/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


NameError: name 'collect_expected' is not defined

In [34]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 7624), started 0:02:06 ago. (Use '!kill 7624' to kill it.)